# DDPG Training for Balancing Robot

This notebook trains a DDPG agent for the balancing robot environment using the provided configurations.

In [ ]:
# Uncomment the following lines to run in Google Colab

# %cd /content
# !git clone https://github.com/EyalPorat/ddpg-balancing-robot.git
# %cd ddpg-balancing-robot
# !git checkout master-organize-python-proj
# %cd /content/ddpg-balancing-robot/python/notebooks

# import sys
# sys.path.append('/content/ddpg-balancing-robot/python')  # Add the repo root to Python path

In [1]:
import sys
sys.path.append('..')

import torch
import numpy as np
from pathlib import Path
import yaml
import matplotlib.pyplot as plt

from src.balancing_robot.models import Actor, Critic, ReplayBuffer, SimNet
from src.balancing_robot.environment import BalancerEnv
from src.balancing_robot.training import DDPGTrainer
from src.balancing_robot.visualization import plot_training_metrics, create_episode_animation

## Load Configurations

In [2]:
# Load DDPG and environment configurations
with open('../configs/ddpg_config.yaml', 'r') as f:
    ddpg_config = yaml.safe_load(f)

with open('../configs/env_config.yaml', 'r') as f:
    env_config = yaml.safe_load(f)

with open('../configs/simnet_config.yaml', 'r') as f:
    simnet_config = yaml.safe_load(f)

# Create log directory
log_dir = Path('logs/ddpg_training')
log_dir.mkdir(parents=True, exist_ok=True)

## Initialize Environment and Models

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set random seeds from config
torch.manual_seed(ddpg_config["training"].get("random_seed", 42))
np.random.seed(ddpg_config["training"].get("random_seed", 42))

# Create environment
env = BalancerEnv(config_path="../configs/env_config.yaml", render_mode="rgb_array")

# Initialize SimNet from config
simnet = SimNet(
    state_dim=env.observation_space.shape[0],
    action_dim=env.action_space.shape[0],
    hidden_dims=simnet_config["model"]["hidden_dims"],
).to(device)

# Load the state dictionary
simnet.load_state_dict(torch.load("logs/simnet_training/simnet_final.pt", map_location=device)["state_dict"])

# Set the simnet in the environment
env.simnet = simnet

# Initialize trainer with config
trainer = DDPGTrainer(env=env, config_path="../configs/ddpg_config.yaml")

# Print model summaries
trainer.print_model_info()

c:\Users\eyalp\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
c:\Users\eyalp\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


Actor(
  (network): Sequential(
    (0): Linear(in_features=6, out_features=8, bias=True)
    (1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Linear(in_features=8, out_features=8, bias=True)
    (4): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (5): ReLU()
  )
  (output_layer): Linear(in_features=8, out_features=1, bias=True)
)
Critic(
  (l1): Linear(in_features=7, out_features=256, bias=True)
  (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
  )
  (output_layer): Linear(in_features=256, out_features=1, bias=True)
)


## Training

In [4]:
# Extract training parameters from config
train_config = ddpg_config['training']

# Train agent
history = trainer.train(
    num_episodes=train_config['total_episodes'],
    max_steps=train_config['max_steps_per_episode'],
    batch_size=train_config['batch_size'],
    eval_freq=train_config['eval_frequency'],
    save_freq=train_config['save_frequency'],
    log_dir=log_dir
)

Episodes:   0%|          | 3/2000 [00:00<01:29, 22.25it/s]c:\Users\eyalp\Documents\PlatformIO\Projects\balancing_robot\python\notebooks\..\src\balancing_robot\training\ddpg_trainer.py:151: UserWarning: Using a target size (torch.Size([512, 512])) that is different to the input size (torch.Size([512, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = torch.nn.functional.mse_loss(current_Q, target_Q)
Episodes:   0%|          | 10/2000 [00:06<28:29,  1.16it/s]

Episode 10: Eval reward = 120.69


Episodes:   1%|          | 20/2000 [00:22<42:04,  1.28s/it]  

Episode 20: Eval reward = -173.78


Episodes:   2%|▏         | 30/2000 [00:27<13:52,  2.37it/s]

...

Episode 750: Eval reward = -3181.75


Episodes:  38%|███▊      | 760/2000 [05:22<08:06,  2.55it/s]

Episode 760: Eval reward = -2399.15


Episodes:  38%|███▊      | 764/2000 [05:24<08:45,  2.35it/s]


KeyboardInterrupt: 

## Analysis and Visualization

In [ ]:
# Plot training metrics
fig = plot_training_metrics(history, save_path=log_dir / 'training_metrics.png')
plt.show()

In [ ]:
# Create demonstration video
def collect_demo_episode(max_steps=500):
    state = env.reset()
    states = []
    total_reward = 0

    for _ in range(max_steps):
        action = trainer.select_action(state, training=False)
        next_state, reward, done, info = env.step(action)
        states.append(state)
        total_reward += reward
        
        if done:
            break
            
        state = next_state
    
    return np.array(states), total_reward

# Collect several episodes and use the best one for visualization
num_episodes = 5
best_reward = float('-inf')
best_states = None

for _ in range(num_episodes):
    states, reward = collect_demo_episode()
    if reward > best_reward:
        best_reward = reward
        best_states = states

print(f"Best episode reward: {best_reward:.2f}")

# Create and display animation
anim = create_episode_animation(
    states=best_states,
    save_path=log_dir / 'demo.mp4'
)
from IPython.display import HTML
HTML(anim.to_jshtml())

## Save Final Model with Metadata

In [ ]:
# Save final model with config and training history
torch.save({
    'actor_state_dict': trainer.actor.state_dict(),
    'critic_state_dict': trainer.critic.state_dict(),
    'training_history': history,
    'config': ddpg_config,
    'env_config': env_config,
    'metadata': {
        'state_dim': env.observation_space.shape[0],
        'action_dim': env.action_space.shape[0],
        'max_action': float(env.action_space.high[0]),
        'final_eval_reward': trainer.evaluate(num_episodes=10)
    }
}, log_dir / 'final_model.pt')

print("Training complete! Model saved with metadata.")